In [ ]:
import numpy as np
import cv2
import glob
from tqdm.notebook import tqdm
import imutils

from numba import jit


In [ ]:
def align_images(image,template,  templateGray, maxFeatures=5000, keepPercent=0.3,
                 debug=False):
    # convert both the input image and template to grayscale
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # use ORB to detect keypoints and extract (binary) local
    # invariant features
    orb = cv2.ORB_create(maxFeatures)
    (kpsA, descsA) = orb.detectAndCompute(imageGray, None)
    (kpsB, descsB) = orb.detectAndCompute(templateGray, None)
    # match the features
    method = cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING
    matcher = cv2.DescriptorMatcher_create(method)
    matches = matcher.match(descsA, descsB, None)
    # sort the matches by their distance (the smaller the distance,
    # the "more similar" the features are)
    matches = sorted(matches, key=lambda x:x.distance)
    # keep only the top matches
    keep = int(len(matches) * keepPercent)
    matches = matches[:keep]
    # check to see if we should visualize the matched keypoints
    matchedVis=None
    if debug:
        matchedVis = cv2.drawMatches(image, kpsA, template, kpsB,
                                     matches, None)
        matchedVis = imutils.resize(matchedVis, width=1000)

    ptsA = np.zeros((len(matches), 2), dtype="float")
    ptsB = np.zeros((len(matches), 2), dtype="float")
    # loop over the top matches
    for (i, m) in enumerate(matches):
        # indicate that the two keypoints in the respective images
        # map to each other
        ptsA[i] = kpsA[m.queryIdx].pt
        ptsB[i] = kpsB[m.trainIdx].pt
    # compute the homography matrix between the two sets of matched
    # points
    (H, mask) = cv2.findHomography(ptsA, ptsB, method=cv2.RANSAC)
    # use the homography matrix to align the images
    (h, w) = template.shape[:2]
    aligned = cv2.warpPerspective(image, H, (w, h))
    # return the aligned image
    return aligned, matchedVis

In [ ]:

files_paths=glob.glob('data/msk22/headers_jpg/*.jpg')
files_paths=[path.replace('\\','/')for path in files_paths]

# save_folder='data/headers_aligned_jpg/'
save_folder='data/msk22/headers_aligned_jpg/'

refFilename = "msk22_ref_header_62263_5.jpg"
template = cv2.imread(refFilename, cv2.IMREAD_COLOR)

templateGray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

for path in tqdm(files_paths):
    if '(' not in path and ')' not in path:
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        try:
            aligned, matchedVis = align_images(image,template, templateGray, debug=False)
            save_name=save_folder+path.split('/')[-1][:-4]
            if matchedVis is not None:
                cv2.imwrite(save_name+'_debug.jpg', matchedVis)
            cv2.imwrite(save_name+'.jpg', aligned)
        except Exception:
            pass